# **NOTATNIK NALEŻY URUCHAMIAĆ NA ŚRODOWISKU** 
Google Colab:
- Z TPU(wolne działanie)
- GPU A100

Kaggle:
- GPU T4x2

W oby przypadkach dostęp do internetu jest wymagany dla 'LLM as a Judge' w quizerze.

# 0. Intalowanie i konfiguracja środowiska, zaciąganie skryptów i plików

### Klonowanie repozytorium z plikami i skryptami dla systemu

In [ ]:
!git clone https://github.com/NukeeMann/rag_mgr.git
!pip install langchain langchain_community morfeusz2 elasticsearch sentence_transformers
!python -m spacy download pl_core_news_sm
!pip install pdfminer.six

### Załadowanie bibliotek

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
import os
import re
import nltk
from nltk.corpus import stopwords
import morfeusz2
from elasticsearch import Elasticsearch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, TextStreamer
import torch
from typing import List
from langchain_core.documents.base import Document
import spacy
from sentence_transformers import util
from tqdm import tqdm
import time
from rag_mgr.notebooks.scripts.rag import RAG
from rag_mgr.notebooks.scripts.quizer import QuizerLLM

nltk.download('stopwords')

!touch /usr/share/nltk_data/corpora/stopwords/polish

# Load polish stopwords
with open('rag_mgr/stopwords-pl.txt', 'r', encoding='utf-8') as file:
    polish_stopwords = file.read().splitlines()

stopwords.words('polish').extend(polish_stopwords)

# 1. Inicjalizacja RAGa

In [ ]:
# Env variables for ElasticSearch database and LLM as a Judge
os.environ['ES_URL'] = 
os.environ['ES_KEY'] = 
os.environ['OPENAI_API_KEY'] = 


# Select ElasticSearch databse
"""
Ze względu, że process wydobywania informacji i umieszczania ich w bazie jest
czasochłonny umieściłem w poniższych indeksach już niektóre dokumenty.
Należy podmienić nazwe indeksu i pominąć następną komórkę z ładowaniem dokumentów.

- kodeks_cywilny <- Baza z kodeksem cywilnym (chunk_size=512, chunk_overlap=64)
- patomorfologia_3_512 <- Baza z trzema podręcznikami (chunk_size=512, chunk_overlap=96):
                        * 'Nauka o Cierpieniu B. Kopycki' 656 str,
                        * 'Patologia Robbins' 966 str,
                        * 'Podstawy Patologii Domagała' 736 str,
- patomorfologia_512 <- Baza z podręcznikiem Nauka O Cierpieniu (chunk_size=512, chunk_overlap=64)
- patomorfologia_1024 <- Baza z podręcznikiem Nauka O Cierpieniu (chunk_size=1024, chunk_overlap=128)
- patomorfologia_1536 <- Baza z podręcznikiem Nauka O Cierpieniu (chunk_size=1536, chunk_overlap=192)
- eti_pg <- Baza z zescrapowanej domeny eti.pg.edu.pl (chunk_size=512, chunk_overlap=64)
- ferdydurke < Baza z książką 'Ferdydurke' Witolda Gombrowicza (chunk_size=256, chunk_overlap=32)

Jeżeli chcemy stworzyć nową baze należy zmienić index_name na unikalny i wykonać
kod z rozdziału '2. Ładowanie danych do bazy'.
"""
index_name = "kodek_cywilny"

# Initiate RAG
rag_system = RAG(index_name)

# 2. Ładowanie danych do bazy

### Processing danych do bazy ElasticSearch

Obsługuje:
- pliki **.txt**
- pliki **.pdf**
- foldery ze strukturą drzewiasta z plikami **.txt .pdf**

In [ ]:
# Directory to load the documents from
docs_root_dir = "path/to/directory/with/documents/"

# Load, split, process and insert documents to the database
rag_system.insert_docs_dir(docs_root_dir, chunk_size=512, chunk_overlap=64)

# 3. Zapytanie do RAGa

In [ ]:
# Question to ask the system
query_text = '''
Udziel odpowiedzi na poniższe dwa pytania:
1. Czy byłeś trenowany na książce Ferdydurke Witolda Gombrowicza?

2. W Ferdydurke Do pojedynku na miny doszło między kim?
'''

# *Optionally* Additional instruction for the generator
additional_instruct = ""

### Generowanie odpowiedzi

In [ ]:
rag_answer = rag_system.infer(query_text, additional_instruct=additional_instruct, use_rag=True, max_new_tokens_v=1000, top_k=3)
print(rag_answer)

# 4. Ewaluacja na pytaniach z Kodeksu Cywilnego i Patomorfologii

## Zdefiniowanie quizera który będzie ewaluował wyniki

In [ ]:
# Define configuration
index_name = "kodeks_cywilny"
questions_path = "rag_mgr/notebooks/questions/KODEKS_CYWILNY_PYTANIA.txt"
additional_information = "Podaj na podstawie którego artykułu Kodeksu Cywilnego wnioskujesz."
top_k = 3

# Change database index
rag_system.change_index(index_name)

# Initiate Evaluator
quizer = QuizerLLM(rag_system, "openai")

##### Ewalucaja pytań z kodeksu cywilnego z wykorzytsaniem RAGa

In [ ]:
quizer.evaluate(questions_path, additional_information, top_k=top_k)

##### Ewaluacja **BEZ** użycia dodatkowych dokumentów

In [ ]:
quizer.evaluate(questions_path, additional_information, use_rag=False)

## Ewaluacja Patomorfologii

In [ ]:
# Define configuration
index_name = "patomorfologia_1536"
questions_path = "rag_mgr/notebooks/questions/PATOMORFOLOGIA_PYTANIA_100_clean.txt"
additional_information = "Twoim jedynym zadaniem jest wskazać jedną z odpowiedzi a) b) c) d) e) która jest poprawną odpowiedzią na pytanie. NIE podawaj wyjaśnień ani uzasadnień."
top_k = 1

# Change database
quizer.set_indexES(index_name)

##### Ewalucaja pytań z patomorfologii z wykorzytsaniem RAGa

In [ ]:
quizer.evaluate(questions_path, additional_information, top_k=top_k)

##### Ewaluacja **BEZ** użycia dodatkowych dokumentów

In [ ]:
quizer.evaluate(questions_path, additional_information, use_rag=False)